In [1]:
import tensorflow as tf
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
%matplotlib inline

In [3]:
from models import ANN_model

model=ANN_model()
# model.summary()

In [4]:
import base64
model.save_weights('test_weights.h5')

In [5]:
with open('test_weights.h5','rb') as file:
    encoded_string = base64.b64encode(file.read())

In [6]:
with open("test_weights_2.h5","wb") as file:
    file.write(base64.b64decode(encoded_string))

In [7]:
model.load_weights('test_weights_2.h5')

In [19]:
import math

In [8]:
with open('test_weights.h5','rb') as file:
    encoded_string = base64.b64encode(file.read())

In [12]:
len(encoded_string)

6998328

In [21]:
step=len(encoded_string)/1000
step=math.ceil(step)

In [24]:
mylist=[]
for i in range(step):
    msg=encoded_string[i*1000:(i+1)*1000]
    mylist.append(msg)

In [26]:
len(mylist[1:])

6998

In [27]:
new_string=mylist[0]
for i in mylist[1:]:
    new_string=new_string+i

In [28]:
len(new_string)

6998328

In [29]:
with open("test_weights_3.h5","wb") as file:
    file.write(base64.b64decode(new_string))

In [30]:
model.load_weights('test_weights_3.h5')

In [15]:
len(model.layers)

23

In [16]:
for i in range(len(model.layers)):
    w1=model.get_layer(index=i).get_weights()
    print(len(w1)) # 9 non-zero

2
0
2
0
0
0
2
0
2
0
0
0
0
2
4
0
0
2
4
0
0
2
0


In [17]:
i=6
w1=model.get_layer(index=i).get_weights()
w1[0].shape #weight

(3, 3, 32, 64)

In [18]:
w1[1].shape #bias

(64,)

In [19]:
# model.layers[0].set_weights(w1)

### select from weights

In [20]:
weights = model.get_weights()
print(weights[0].shape)  #layer1's weights
print(weights[1].shape)  #layer1's bias
len(weights) #why???? shouldn't it be 18???

(3, 3, 3, 32)
(32,)


22

In [21]:
for i in range(len(weights)):
    print(weights[i].shape)

(3, 3, 3, 32)
(32,)
(3, 3, 32, 32)
(32,)
(3, 3, 32, 64)
(64,)
(3, 3, 64, 64)
(64,)
(2304, 512)
(512,)
(512,)
(512,)
(512,)
(512,)
(512, 100)
(100,)
(100,)
(100,)
(100,)
(100,)
(100, 10)
(10,)


In [22]:
len(weights) ## since 

22

### Set only the final layer trainable

In [23]:
model.trainable = False

for i in range(3):
    model.layers[-(i+1)].trainable = True